# Issue Credential - Carol
## Role: Issuer

### This notebook walks through how to issue a credential across a previously established connection. It should be run in parallel with the [Holder notebook](http://localhost:8889/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb) controlling the agent receiving the credential.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


## 1. Initialise a controller for Carol

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "carol_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
    

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)


Subscribing too: issue_credential


## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)
* [Bob](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)

In [17]:
# remove = await agent_controller.connections.remove_connection('2cb9f84e-2cc5-49c2-8ffd-2c7ca5241f66')

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    


ClientConnectorError: Cannot connect to host carol-agent:8081 ssl:default [Name or service not known]

Error during GET /connections: Cannot connect to host carol-agent:8081 ssl:default [Name or service not known]


## 4. Write a Schema to the Ledger

**Note: You will only be able to do this once unless you change the schema_name or version. Or tear down the current ledger using ./manage down**

In [19]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_contributor"
# Can version the schema if you wish to update it
schema_version = "0.0.2"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.2


## 5. Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/lab/tree/2%20Credentials/Part%201%20-%20Credential%20Schema%20and%20Definitions.ipynb)

**Note: Again this can only be done once per issuer, per schema_id.**

In [20]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:18:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

## 6. Populate the Attribues to Issue to Bob

The schema defines two attributes: name and skill. Both must be populated with attribute values that Alice wishes to issue in a credential to Bob. To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [6]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": "Alice"},
    {"name": "name", "value": "researcher"},
    {"name": "time", "value": time},
    {"name": "is_original", "value": "1"}
]
print(credential_attributes)

[{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]


## 7. Continue to the [Holder Notebook](http://localhost:8889/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb)

It works best if this notebook is already initialised and has listeners registered.


## 9. Send Credential

This sends a credential to a holder (Bob), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False.

In [11]:
schema_id = 'PQRXDxdGqQGSZ8z69p4xZP:2:chloebaba_schema:0.0.1'
cred_def_id = 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:29:default'
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


Credential exchange 3db98c59-9737-430d-b36f-be65c445fb44, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 3db98c59-9737-430d-b36f-be65c445fb44, role: issuer, state: offer_sent
Offering: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]
Handle Credentials
Credential exchange 1a197402-0bbb-4077-94f3-524146f8fe37, role: issuer, state: request_received
Offering: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]
Handle Credentials
Credential exchange 1a197402-0bbb-4077-94f3-524146f8fe37, role: issuer, state: credential_issued
Offering: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]
Handle Credentials
Credential 

## 10. Continue on Holder Notebook



## Misc Api Calls (Optional)

## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [23]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'updated_at': '2023-01-01 15:44:50.936856Z', 'trace': True, 'credential_request': {'prover_did': '6UJzVcKRUTNd2Z3fkgSwqc', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:18:default', 'blinded_ms': {'u': '82525681500742288148474564630177094233184980851380856385643646115889543522307497349840857727974165807714876639783314434724269466393656929818284628647996398296674111609500398019777169749657303890511154419364730801076464000352467829667886405771568819889301124620633944121384990396825799437590653771834588795191024056073949877528822961161747036495762828773630735259939702681590283910085724268648305287085757593665064635779190223589222401334448927470481882500872630149291408078786711799774300892960067066157746834923247479416365729865676682129161874348395763152577920576060404243726449366520655263685500466808013724930270', 'ur': None, 'hidden_attributes': ['master_secret'], 'committed_attributes': {}}, 'blinded_ms_correctness_proof': {'c': '802916906344187494726718614632532122177072667567705308534

In [12]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 3db98c59-9737-430d-b36f-be65c445fb44, role: issuer, state: offer_sent


## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?). 

**Note: For some usecases keeping these exchange records may be very important. E.g. For audit purposes.**

In [ ]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

In [ ]:
response = await agent_controller.issuer.get_records()
print(response)

### TODO: Document the api calls not used. 

* Send_offer
* send_offer_for_record
* issue_credential - If rather than send_credential the notebook used send_offer then the entity controlling the agent (you with this notebook and agent_controller api) would have to explicitly issue the credential to once Bob requested it. With send_credential this part is automated.

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [6]:
response = await agent_controller.terminate()
print(response)

None
